In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import torch
import json

from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from diveslowlearnfast.train import helper as train_helper
from diveslowlearnfast.models import SlowFast, load_checkpoint
from diveslowlearnfast.config import Config

from diveslowlearnfast.transforms import CenterCropVideo
from torchvision.transforms._transforms_video import NormalizeVideo
from pytorchvideo.transforms import UniformTemporalSubsample, ShortSideScale
from torchvision.transforms.v2 import Compose, Lambda
from pytorchvideo.data.encoded_video import EncodedVideo

import shap

/Users/youritomassen/Projects/Uni/Master Thesis/diveslowlearnfast/.venv/lib/python3.12/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/Users/youritomassen/Projects/Uni/Master Thesis/diveslowlearnfast/.venv/lib/python3.12/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [2]:
class DualInputWrapper(torch.nn.Module):
    def __init__(self, model):
        super(DualInputWrapper, self).__init__()
        self.model = model


    def forward(self, xb):
        x_fast = xb[:]
        x_slow = x_fast[:, :, ::cfg.SLOWFAST.ALPHA]
        input = [x_slow, x_fast]
        return self.model(input)

cfg = Config()
cfg.DATA.DATASET_PATH = '/Users/youritomassen/Projects/xai/data/Diving48/'
cfg.TRAIN.BATCH_SIZE = 24
cfg.DATA.NUM_FRAMES = 8
cfg.DATA.TRAIN_CROP_SIZE = 224
cfg.DATA.TEST_CROP_SIZE = 224
cfg.MODEL.INPLACE_RELU = False
device = torch.device('cpu')
model = SlowFast(cfg)
_, optimiser, train_loader, *_ = train_helper.get_train_objects(cfg, model)
model, *_ = load_checkpoint(model, optimiser, '../misc/checkpoint.pth', device)
model = DualInputWrapper(model)

test_loader = train_helper.get_test_objects(cfg)

In [3]:
baseline_values, *_ = next(iter(train_loader))
baseline_values.shape

torch.Size([24, 3, 8, 224, 224])

In [4]:
x, y, *_ = next(iter(test_loader))
x.shape

torch.Size([24, 3, 8, 224, 224])

In [6]:
e = shap.DeepExplainer(model, baseline_values)
shap_values = e.shap_values(x)

KeyboardInterrupt: 